Формирование JSON на оснве текстового документа с ПДД в РФ
---

In [1]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.9 MB/s eta 0:00:00


In [5]:
import re
import json
from docx import Document

def read_docx_file(file_path):
    """Чтение .docx файла и извлечение текста"""
    doc = Document(file_path)
    full_text = []

    for paragraph in doc.paragraphs:
        if paragraph.text.strip():  # Игнорируем пустые строки
            full_text.append(paragraph.text)

    return '\n'.join(full_text)

def parse_pdd_to_json(text):
    lines = text.split('\n')

    result = []
    current_section = None
    current_content = []
    current_main_point = None
    current_sub_points = []

    for line in lines:
        line = line.strip()
        if not line:
            continue

        section_match = re.match(r'^(\d+)\.\s+(.+)$', line)
        if section_match:
            if current_section is not None:
                save_current_section(current_section, current_content, current_main_point, current_sub_points, result)

            current_section = {
                'num': section_match.group(1),
                'name': section_match.group(2).strip()
            }
            current_content = []
            current_main_point = None
            current_sub_points = []
            continue

        main_p_match = re.match(r'^(\d+\.\d+)\.\s+(.+)$', line)
        if main_p_match and current_section is not None:
            if current_main_point is not None:
                save_current_main_point(current_content, current_main_point, current_sub_points)

            current_main_point = {
                'p_num': main_p_match.group(1),
                'full_text': main_p_match.group(2).strip(),
                'text_buffer': [main_p_match.group(2).strip()]
            }
            current_sub_points = []
            continue

        sub_p_match = re.match(r'^(\d+\.\d+\.\d+)\.\s+(.+)$', line)
        if sub_p_match and current_section is not None and current_main_point is not None:
            if not current_sub_points and current_main_point['text_buffer']:
                current_main_point['full_text'] = ' '.join(current_main_point['text_buffer']).strip()

            sub_point = {
                'p_num': sub_p_match.group(1),
                'full_text': sub_p_match.group(2).strip(),
                'text_buffer': [sub_p_match.group(2).strip()]
            }
            current_sub_points.append(sub_point)
            continue

        if current_main_point is not None and line:
            if current_sub_points:
                current_sub_points[-1]['text_buffer'].append(line)
            else:
                current_main_point['text_buffer'].append(line)

    if current_section is not None:
        save_current_section(current_section, current_content, current_main_point, current_sub_points, result)

    return result

def save_current_main_point(content_list, main_point, sub_points):
    """Сохраняет основной пункт с подпунктами в структуру"""
    if main_point is None:
        return

    if main_point['text_buffer']:
        main_point['full_text'] = ' '.join(main_point['text_buffer']).strip()

    point_structure = {
        'p_num': main_point['p_num'],
        'full_text': main_point['full_text']
    }

    if sub_points:
        for sub_point in sub_points:
            if sub_point['text_buffer']:
                sub_point['full_text'] = ' '.join(sub_point['text_buffer']).strip()

        point_structure['p_sup'] = [
            {
                'p_num': sub_point['p_num'],
                'full_text': sub_point['full_text']
            }
            for sub_point in sub_points
        ]

    content_list.append(point_structure)

def save_current_section(section, content_list, main_point, sub_points, result_list):
    """Сохраняет текущий раздел в результат"""
    if section is None:
        return

    if main_point is not None:
        save_current_main_point(content_list, main_point, sub_points)

    if content_list:
        section_data = {
            'section': {
                'sec_num': section['num'],
                'name': section['name'],
                'content': content_list
            }
        }
        result_list.append(section_data)

def process_pdd_file(input_file, output_file):
    try:
        print(f"Чтение файла: {input_file}")
        content = read_docx_file(input_file)

        print("Парсинг содержимого...")
        parsed_data = parse_pdd_to_json(content)

        print(f"Сохранение в: {output_file}")
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(parsed_data, f, ensure_ascii=False, indent=2)

        return parsed_data

    except Exception as e:
        print(f"Ошибка при обработке: {e}")
        return None

def print_statistics(parsed_data):
    """Выводит статистику по обработанным данным"""
    if not parsed_data:
        print("Нет данных для статистики")
        return

    print(f"Найдено разделов: {len(parsed_data)}")

    total_main_points = 0
    total_sub_points = 0
    sections_with_subpoints = 0

    for section_data in parsed_data:
        section = section_data['section']
        main_points_count = len(section['content'])
        total_main_points += main_points_count

        sub_points_in_section = 0
        for point in section['content']:
            if 'p_sup' in point:
                sub_points_count = len(point['p_sup'])
                total_sub_points += sub_points_count
                sub_points_in_section += sub_points_count

        if sub_points_in_section > 0:
            sections_with_subpoints += 1

        print(f"Раздел {section['sec_num']}. {section['name']}:")
        print(f"  - Основных пунктов: {main_points_count}")
        print(f"  - Подпунктов: {sub_points_in_section}")

    print(f"\nИтоговая статистика:")
    print(f"Всего основных пунктов: {total_main_points}")
    print(f"Всего подпунктов: {total_sub_points}")
    print(f"Разделов с подпунктами: {sections_with_subpoints}")

def print_example_structure(parsed_data, section_index=0, point_index=0):
    """Выводит пример структуры для наглядности"""
    if not parsed_data or section_index >= len(parsed_data):
        print("Нет данных для примера")
        return

    section = parsed_data[section_index]['section']
    print(f"\nПример структуры раздела {section['sec_num']}. {section['name']}:")

    if section['content'] and point_index < len(section['content']):
        point = section['content'][point_index]
        print(f"Основной пункт: {point['p_num']}")
        print(f"Текст: {point['full_text'][:100]}...")

        if 'p_sup' in point:
            print(f"Подпункты ({len(point['p_sup'])}):")
            for sub_point in point['p_sup'][:2]:
                print(f"  - {sub_point['p_num']}: {sub_point['full_text'][:80]}...")
        else:
            print("Подпункты: нет")


if __name__ == "__main__":
    input_filename = "/content/пдд.docx"
    output_filename = "pdd_structured.json"

    result = process_pdd_file(input_filename, output_filename)

    if result:
        print(f"Успешно обработано! Результат сохранен в {output_filename}")
        print_statistics(result)
        print_example_structure(result)
        simplified_output = "pdd_simplified_example.json"
        with open(simplified_output, 'w', encoding='utf-8') as f:
            simplified_data = result[:2] if len(result) >= 2 else result
            json.dump(simplified_data, f, ensure_ascii=False, indent=2)
        print(f"\nУпрощенный пример сохранен в: {simplified_output}")

    else:
        print("Ошибка при обработке файла")

Чтение файла: /content/пдд.docx
Парсинг содержимого...
Сохранение в: pdd_structured.json
Успешно обработано! Результат сохранен в pdd_structured.json
Найдено разделов: 30
Раздел 2. Общие обязанности водителей:
  - Основных пунктов: 7
  - Подпунктов: 6
Раздел 3. Применение специальных сигналов:
  - Основных пунктов: 6
  - Подпунктов: 0
Раздел 4. Обязанности пешеходов:
  - Основных пунктов: 8
  - Подпунктов: 0
Раздел 5. Обязанности пассажиров:
  - Основных пунктов: 2
  - Подпунктов: 0
Раздел 6. Сигналы светофора и регулировщика:
  - Основных пунктов: 16
  - Подпунктов: 0
Раздел 7. Применение аварийной сигнализации и знака аварийной остановки:
  - Основных пунктов: 3
  - Подпунктов: 0
Раздел 8. Начало движения, маневрирование:
  - Основных пунктов: 12
  - Подпунктов: 0
Раздел 9. Расположение транспортных средств на проезжей части:
  - Основных пунктов: 12
  - Подпунктов: 0
Раздел 10. Скорость движения:
  - Основных пунктов: 5
  - Подпунктов: 0
Раздел 11. Обгон, опережение, встречный разъе

После собранный JSON был прогнан через LLM c целью формирования дополнительнвх полей с описанием содержимого каждой секции для повышения качества дальнейшего поиска на уровне секций ПДД

Вариант faiss по итоговому тексту с трехуровневым поиском:
---
- секция
- пункт
- подпункт

плюс создан игрушечный поисковик, позволяющий оценить качество выполняемого поиска и ранжирование итоговой выдачи



In [4]:
!pip install faiss-cpu sentence-transformers numpy scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 83.8 MB/s eta 0:00:00


In [ ]:
!pip install langchain langchain-community faiss-cpu sentence-transformers langchain-huggingface

In [3]:
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from typing import List, Dict, Tuple, Any
import re
import pickle
import os
from datetime import datetime

class InteractiveThreeLevelSearch:
    def __init__(self, json_file_path, model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'):
        self.json_file_path = json_file_path
        self.model = SentenceTransformer(model_name)
        self.sections = []
        self.section_texts = []  # Объединенные name + description
        self.section_embeddings = None
        self.section_index = None
        self.content_data = []
        self.content_embeddings = None
        self.content_index = None
        self.qa_history = []
        self.section_structure_cache = {}

    def load_data(self):
        """Загрузка данных из JSON файла"""
        print("📂 Загрузка данных ПДД...")
        with open(self.json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        self.sections = []
        self.section_texts = []  # name + description
        self.content_data = []

        for section_item in data:
            section = section_item['section']
            sec_num = section['sec_num']
            name = section['name']
            description = section['description']


            section_text = f"{name}. {description}"

            self.sections.append({
                'sec_num': sec_num,
                'name': name,
                'description': description,
                'section_text': section_text,
                'original_data': section
            })
            self.section_texts.append(section_text)

            section_idx = len(self.sections) - 1
            if 'content' in section:
                for content_item in section['content']:
                    p_num = content_item['p_num']
                    full_text = content_item['full_text']

                    self.content_data.append({
                        'section_idx': section_idx,
                        'p_num': p_num,
                        'full_text': full_text,
                        'is_subpoint': False,
                        'parent_p_num': None,
                        'original_data': content_item
                    })

                    if 'p_sup' in content_item:
                        for subpoint in content_item['p_sup']:
                            sub_p_num = subpoint['p_num']
                            sub_full_text = subpoint['full_text']

                            self.content_data.append({
                                'section_idx': section_idx,
                                'p_num': sub_p_num,
                                'full_text': sub_full_text,
                                'is_subpoint': True,
                                'parent_p_num': p_num,
                                'original_data': subpoint
                            })

        print(f"✅ Загружено {len(self.sections)} разделов и {len(self.content_data)} пунктов/подпунктов")

    def build_indexes(self):
        """Построение FAISS индексов с нормализацией ВСЕХ векторов"""
        print("🔨 Построение векторных индексов...")


        if self.section_texts:
            section_embeddings = self.model.encode(self.section_texts)
            faiss.normalize_L2(section_embeddings)
            self.section_embeddings = section_embeddings.astype('float32')
            self.section_index = faiss.IndexFlatIP(self.section_embeddings.shape[1])
            self.section_index.add(self.section_embeddings)
            print(f"✅ Построен индекс разделов: {len(self.section_texts)} векторов (name + description)")


        content_texts = [item['full_text'] for item in self.content_data]
        if content_texts:
            content_embeddings = self.model.encode(content_texts)
            faiss.normalize_L2(content_embeddings)
            self.content_embeddings = content_embeddings.astype('float32')
            self.content_index = faiss.IndexFlatIP(self.content_embeddings.shape[1])
            self.content_index.add(self.content_embeddings)
            print(f"✅ Построен индекс пунктов: {len(content_texts)} векторов")

        print("🎯 Все векторы нормализованы для косинусного сходства")

    def save_indexes(self, cache_dir):
        """Сохранение индексов в кэш"""
        try:
            if not os.path.exists(cache_dir):
                os.makedirs(cache_dir)

            faiss.write_index(self.sections_index, os.path.join(cache_dir, 'sections_index.faiss'))
            faiss.write_index(self.content_index, os.path.join(cache_dir, 'content_index.faiss'))

            metadata = {
                'sections': self.sections,
                'section_texts': self.section_texts,
                'section_embeddings': self.section_embeddings,
                'content_data': self.content_data,
                'content_embeddings': self.content_embeddings
            }

            with open(os.path.join(cache_dir, 'metadata.pkl'), 'wb') as f:
                pickle.dump(metadata, f)

            print("✅ Индексы успешно сохранены в кэш")

        except Exception as e:
            print(f"❌ Ошибка сохранения индексов: {e}")

    def load_indexes(self, cache_dir):
      """Загрузка индексов из кэша"""
      try:
          if not os.path.exists(cache_dir):
              return False

          index_files = [
              os.path.join(cache_dir, 'sections_index.faiss'),
              os.path.join(cache_dir, 'content_index.faiss')
          ]

          if not all(os.path.exists(f) for f in index_files):
              return False

          self.sections_index = faiss.read_index(index_files[0])
          self.content_index = faiss.read_index(index_files[1])

          with open(os.path.join(cache_dir, 'metadata.pkl'), 'rb') as f:
              data = pickle.load(f)

          self.sections = data.get('sections', [])
          self.section_texts = data.get('section_texts', [])
          self.section_embeddings = data.get('section_embeddings', [])
          self.content_data = data.get('content_data', [])
          self.content_embeddings = data.get('content_embeddings', [])

          print("✅ Индексы успешно загружены из кэша")
          return True

      except Exception as e:
          print(f"❌ Ошибка загрузки индексов: {e}")
          return False

    def initialize(self, force_rebuild=False):
        """Инициализация системы поиска"""
        cache_dir = "faiss_cache"

        if not force_rebuild and self.load_indexes(cache_dir):
            return True

        print("🔨 Построение индексов с нуля...")
        self.load_data()
        self.build_indexes()
        self.save_indexes(cache_dir)
        return True

    def search(self, query: str, top_k_sections: int = 3, top_k_content: int = 10, threshold: float = 0.3) -> List[Dict]:
        """Трехуровневый поиск с учетом name и description разделов"""

        # Уровень 1: Поиск по разделам (name + description)
        query_embedding = self.model.encode([query]).astype('float32')
        faiss.normalize_L2(query_embedding)

        section_scores, section_indices = self.section_index.search(query_embedding, top_k_sections)

        results = []

        # Уровень 2 и 3: Поиск в найденных разделах
        for section_score, section_idx in zip(section_scores[0], section_indices[0]):
            if section_score < threshold:
                continue

            section_info = self.sections[section_idx]

            section_content_indices = [
                i for i, item in enumerate(self.content_data)
                if item['section_idx'] == section_idx
            ]

            if not section_content_indices:
                continue

            section_content_embeddings = self.content_embeddings[section_content_indices]

            temp_index = faiss.IndexFlatIP(section_content_embeddings.shape[1])
            temp_index.add(section_content_embeddings)

            content_scores, content_local_indices = temp_index.search(
                query_embedding,
                min(top_k_content, len(section_content_indices))
            )

            section_results = []
            for content_score, local_idx in zip(content_scores[0], content_local_indices[0]):
                if content_score < threshold:
                    continue

                global_idx = section_content_indices[local_idx]
                content_item = self.content_data[global_idx]


                # Преобразуем в [0, 1] для удобства отображения
                section_score_norm = (section_score + 1) / 2
                content_score_norm = (content_score + 1) / 2
                combined_score = section_score_norm * content_score_norm

                section_results.append({
                    'p_num': content_item['p_num'],
                    'full_text': content_item['full_text'],
                    'is_subpoint': content_item['is_subpoint'],
                    'parent_p_num': content_item['parent_p_num'],
                    'section_score': float(section_score),
                    'content_score': float(content_score),
                    'combined_score': combined_score,
                    'section_score_norm': section_score_norm,
                    'content_score_norm': content_score_norm
                })

            section_results.sort(key=lambda x: (x['section_score_norm'], x['content_score_norm'], x['combined_score']), reverse=True)

            if section_results:
                results.append({
                    'section': {
                        'sec_num': section_info['sec_num'],
                        'name': section_info['name'],
                        'description': section_info['description'],
                        'section_text': section_info['section_text'],
                        'score': float(section_score),
                        'score_norm': section_score_norm
                    },
                    'content': section_results
                })

        results.sort(key=lambda x: max([item['section_score_norm'] for item in x['content']]), reverse=True)

        return results

    def _parse_p_num(self, p_num: str) -> Tuple[int, ...]:
        """Парсинг номеров пунктов для корректной сортировки"""
        parts = re.findall(r'\d+', p_num)
        return tuple(int(part) for part in parts)

    def search_and_display(self, query: str):
        """Поиск и отображение результатов"""
        print(f"\n🔍 Поиск: '{query}'")
        print("=" * 80)

        start_time = datetime.now()
        results = self.search(query, top_k_sections=3, top_k_content=5, threshold=0.2)
        search_time = (datetime.now() - start_time).total_seconds()

        total_results = sum(len(result['content']) for result in results)

        self.qa_history.append({
            'query': query,
            'timestamp': datetime.now(),
            'results_count': total_results,
            'search_time': search_time
        })

        if not results:
            print("❌ По вашему запросу ничего не найдено")
            print("💡 Попробуйте использовать другие ключевые слова")
            return

        print(f"✅ Найдено {total_results} результатов за {search_time:.2f} сек")
        print("📊 Поиск по разделам учитывает: Название + Описание")
        print()

        for i, result in enumerate(results, 1):
            section = result['section']
            print(f"📁 РАЗДЕЛ {i}: {section['sec_num']}. {section['name']}")
            print(f"   📝 Описание: {section['description']}")
            print(f"   🎯 Сходство раздела: {section['score']:.3f}")
            print()

            for j, content in enumerate(result['content'], 1):
                indent = "    "
                prefix = f"{content['p_num']}"
                if content['is_subpoint']:
                    prefix = f"  ↳ {content['p_num']}"

                print(f"{indent}{prefix}")
                text_preview = content['full_text'][:150] + "..." if len(content['full_text']) > 150 else content['full_text']
                print(f"{indent}   {text_preview}")
                print(f"{indent}   🎯 Сходство: раздел={content['section_score']:.3f}, пункт={content['content_score']:.3f}")
                print(f"{indent}   📈 Комбинированный score: {content['combined_score']:.3f}")
                print()

            print("-" * 80)

    def explore_section(self, section_num: str):
        """Исследование структуры раздела"""
        print(f"\n🔍 Исследование раздела {section_num}")
        print("=" * 60)

        section_found = None
        for section in self.sections:
            if section['sec_num'] == section_num:
                section_found = section
                break

        if not section_found:
            print(f"❌ Раздел {section_num} не найден")
            return

        print(f"📁 РАЗДЕЛ {section_found['sec_num']}: {section_found['name']}")
        print(f"📝 {section_found['description']}")
        print(f"🔍 Текст для поиска: {section_found['section_text']}")
        print()

        section_content = [
            item for item in self.content_data
            if item['section_idx'] == self.sections.index(section_found)
        ]

        main_points = {}
        for item in section_content:
            if not item['is_subpoint']:
                main_points[item['p_num']] = {
                    'item': item,
                    'subpoints': []
                }

        for item in section_content:
            if item['is_subpoint'] and item['parent_p_num'] in main_points:
                main_points[item['parent_p_num']]['subpoints'].append(item)


        for p_num in sorted(main_points.keys(), key=self._parse_p_num):
            main_point = main_points[p_num]
            print(f"📄 {p_num}: {main_point['item']['full_text']}")

            for subpoint in sorted(main_point['subpoints'], key=lambda x: self._parse_p_num(x['p_num'])):
                print(f"   ↳ {subpoint['p_num']}: {subpoint['full_text']}")

            print()

    def show_search_stats(self):
        """Показать статистику поиска"""
        if not self.qa_history:
            print("📊 История поиска пуста")
            return

        print(f"\n📊 СТАТИСТИКА ПОИСКА")
        print("=" * 40)
        print(f"Всего запросов: {len(self.qa_history)}")

        recent_queries = self.qa_history[-10:] if len(self.qa_history) > 10 else self.qa_history
        avg_results = sum(item['results_count'] for item in recent_queries) / len(recent_queries)
        avg_time = sum(item['search_time'] for item in recent_queries) / len(recent_queries)

        print(f"Среднее количество результатов: {avg_results:.1f}")
        print(f"Среднее время поиска: {avg_time:.2f} сек")
        print(f"Размер базы: {len(self.sections)} разделов, {len(self.content_data)} пунктов")
        print(f"Поиск по разделам: Название + Описание")

        print(f"\n🎯 Последние запросы:")
        for i, item in enumerate(self.qa_history[-5:], 1):
            print(f"  {i}. '{item['query']}' - {item['results_count']} результатов")

def interactive_three_level_search():
    """Интерактивный режим с трехуровневым поиском"""
    search_system = InteractiveThreeLevelSearch('pdd.json')
    search_system.initialize(force_rebuild=False)

    print("🎯 ТРЕХУРОВНЕВЫЙ ПОИСК ПО ПДД")
    print("=" * 50)
    print("Уровни поиска: Разделы → Пункты → Подпункты")
    print("Ранжирование: Раздел × Пункт × Подпункт")
    print("Поиск по разделам: Название + Описание")
    print("\nКоманды:")
    print("  'раздел X' - посмотреть структуру раздела X")
    print("  'история' - показать историю поиска")
    print("  'статистика' - показать статистику")
    print("  'выход' - завершить работу")
    print()

    while True:
        try:
            query = input("\n🔍 Введите вопрос по ПДД: ").strip()

            if query.lower() in ['выход', 'exit', 'quit']:
                print("\n👋 До свидания!")
                break

            if query.lower().startswith('раздел '):
                section_num = query[7:].strip()
                search_system.explore_section(section_num)
                continue

            if query.lower() == 'история':
                print(f"\n📊 История поиска: {len(search_system.qa_history)} запросов")
                for i, item in enumerate(search_system.qa_history[-5:], 1):
                    print(f"  {i}. '{item['query']}' - {item['results_count']} результатов")
                continue

            if query.lower() == 'статистика':
                search_system.show_search_stats()
                continue

            if len(query) < 2:
                print("❌ Слишком короткий запрос")
                continue

            search_system.search_and_display(query)

        except KeyboardInterrupt:
            print("\n\n👋 До свидания!")
            break
        except Exception as e:
            print(f"❌ Ошибка: {e}")


def install_dependencies():
    """Функция для установки необходимых зависимостей"""
    try:
        import faiss
        from sentence_transformers import SentenceTransformer
        print("✅ Все зависимости установлены")
        return True
    except ImportError as e:
        print("❌ Необходимо установить зависимости:")
        print("pip install faiss-cpu sentence-transformers")
        return False

if __name__ == "__main__":
    print("🚀 Запуск системы поиска по ПДД...")
    if install_dependencies():
        interactive_three_level_search()

🚀 Запуск системы поиска по ПДД...
✅ Все зависимости установлены
🔨 Построение индексов с нуля...
📂 Загрузка данных ПДД...
✅ Загружено 24 разделов и 222 пунктов/подпунктов
🔨 Построение векторных индексов...
✅ Построен индекс разделов: 24 векторов (name + description)
✅ Построен индекс пунктов: 222 векторов
🎯 Все векторы нормализованы для косинусного сходства
❌ Ошибка сохранения индексов: 'InteractiveThreeLevelSearch' object has no attribute 'sections_index'
🎯 ТРЕХУРОВНЕВЫЙ ПОИСК ПО ПДД
Уровни поиска: Разделы → Пункты → Подпункты
Ранжирование: Раздел × Пункт × Подпункт
Поиск по разделам: Название + Описание

Команды:
  'раздел X' - посмотреть структуру раздела X
  'история' - показать историю поиска
  'статистика' - показать статистику
  'выход' - завершить работу


🔍 Введите вопрос по ПДД: скорость разрешенная для движения

🔍 Поиск: 'скорость разрешенная для движения'
✅ Найдено 15 результатов за 0.03 сек
📊 Поиск по разделам учитывает: Название + Описание

📁 РАЗДЕЛ 1: 9. Скорость движения